In [1]:
email_conversation = """From: 테디 (teddy@teddynote.com)
To: 이은채 대리님 (eunchae@teddyinternational.me)
Subject: RAG 솔루션 시연 관련 미팅 제안

안녕하세요, 이은채 대리님,

저는 테디노트의 테디입니다. 최근 귀사에서 AI를 활용한 혁신적인 솔루션을 모색 중이라는 소식을 들었습니다. 테디노트는 AI 및 RAG 솔루션 분야에서 다양한 경험과 노하우를 가진 기업으로, 귀사의 요구에 맞는 최적의 솔루션을 제공할 수 있다고 자부합니다.

저희 테디노트의 RAG 솔루션은 귀사의 데이터 활용을 극대화하고, 실시간으로 정확한 정보 제공을 통해 비즈니스 의사결정을 지원하는 데 탁월한 성능을 보입니다. 이 솔루션은 특히 다양한 산업에서의 성공적인 적용 사례를 통해 그 효과를 입증하였습니다.

귀사와의 협력 가능성을 논의하고, 저희 RAG 솔루션의 구체적인 기능과 적용 방안을 시연하기 위해 미팅을 제안드립니다. 다음 주 목요일(7월 18일) 오전 10시에 귀사 사무실에서 만나 뵐 수 있을까요?

미팅 시간을 조율하기 어려우시다면, 편하신 다른 일정을 알려주시면 감사하겠습니다. 이은채 대리님과의 소중한 만남을 통해 상호 발전적인 논의가 이루어지길 기대합니다.

감사합니다.

테디
테디노트 AI 솔루션팀"""

In [21]:
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field


class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    company: str = Field(description="메일을 보낸 사람의 회사")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

In [22]:
from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [23]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=output_parser.get_format_instructions())

In [24]:
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [25]:
chain = prompt | llm | output_parser

In [37]:
answer = chain.invoke(
    {"email_conversation": email_conversation, "question": "이메일 내용을 분석해줘"}
)

In [39]:
answer

EmailSummary(person='테디', email='teddy@teddynote.com', company='테디노트', subject='RAG 솔루션 시연 관련 미팅 제안', summary='테디노트의 테디가 이은채 대리님에게 RAG 솔루션 시연을 위한 미팅을 제안하는 이메일입니다. 테디노트는 AI 및 RAG 솔루션 분야에서 다양한 경험과 노하우를 가지고 있으며, 귀사의 데이터 활용을 극대화하고 실시간으로 정확한 정보를 제공하는 솔루션을 제공합니다. 미팅은 다음 주 목요일(7월 18일) 오전 10시에 제안되었으며, 다른 일정이 필요할 경우 조율을 요청하고 있습니다.', date='다음 주 목요일(7월 18일) 오전 10시')

참고: https://serpapi.com/integrations/python

In [29]:
import os

os.environ["SERPAPI_API_KEY"] = (
    "e76de14ee240e0051ed8bb05d5db568dd1dc9cfcaa2b51fd83613829a85bf244"
)

In [62]:
from langchain_community.utilities import SerpAPIWrapper

params = {
    "engine": "google",
    "gl": "kr",
    "hl": "ko",
}

search = SerpAPIWrapper(params=params)

In [63]:
print(search.run(f"{answer.person} {answer.company} {answer.email}"))

['테디노트 데이터와 인공지능을 좋아하는 개발자 노트 · 검색. 토글 메뉴. 카테고리 · 태그 · 연도 · 강의 · 어바웃미 · Teddy. Creator & Data Lover. 팔로우. Pangyo, ...', '테디노트 X 패스트캠퍼스 "RAG 비법노트" · 환경 설정 (Mac) · 환경 설정 ... TeddyNote. Enter to select. to navigate. ESC to close. 테디노트.dev. Teddy Lee (이경록) ...', '테디노트 TeddyNote. @teddynote. 23.3K subscribers•182 videos. 데이터 분석, 머신러닝, 딥러닝, LLM 에 대한 내용을 다룹니다. 연구보다는 개발에 관심이 많습니다 \u200d ...', '테디노트 YouTube & Blog ▷ 랭체인 한국어 튜토리얼 저자(https://wikidocs.net/book/14314) ▷ Github: https://github.com/teddylee777', '✔️ 코드스테이츠 X 테디노트 - 깃헙 블로그 제작하기 강의. ✔️ SK그룹 - 2023년 상반기 텐서플로우 딥러닝 과정 강의. ✔️ S-Oil - 파이썬 데이터 분석, 머신러닝 과정 강의 ...', 'Few Shot Prompt Template · yaml 형태로 관리하는 것을 추천! · 나중에 프롬프트 유지보수 관리에 용이 · 파일명을 신경써서 작성하기.', '데이터 분석, 머신러닝, 딥러닝, LLM 에 대한 내용을 다룹니다. 연구보다는 개발에 관심이 많습니다 \u200d♂️ ...more ...more fastcampus.co.kr/data_online_teddyand 2 ...', '안녕하세요? 두 번째 발표를 맡은 테디노트의 이경록(Teddy)입니다. 먼저, 귀중한 자리에 초청해 주신 랭체인 코리아 운영진 및 관계자 여러분께 감사 ...', '교육 스타트업 브레인크루 주식회사 설립, CEO. 특허출원 10-2019-0058830 (인공지능 기반 강의 추천 및 학습 동

In [65]:
search_result = search.run(f"{answer.person} {answer.company} {answer.email}")
print(search_result)

['테디노트 데이터와 인공지능을 좋아하는 개발자 노트 · 검색. 토글 메뉴. 카테고리 · 태그 · 연도 · 강의 · 어바웃미 · Teddy. Creator & Data Lover. 팔로우. Pangyo, ...', '테디노트 X 패스트캠퍼스 "RAG 비법노트" · 환경 설정 (Mac) · 환경 설정 ... TeddyNote. Enter to select. to navigate. ESC to close. 테디노트.dev. Teddy Lee (이경록) ...', '테디노트 TeddyNote. @teddynote. 23.3K subscribers•182 videos. 데이터 분석, 머신러닝, 딥러닝, LLM 에 대한 내용을 다룹니다. 연구보다는 개발에 관심이 많습니다 \u200d ...', '테디노트 YouTube & Blog ▷ 랭체인 한국어 튜토리얼 저자(https://wikidocs.net/book/14314) ▷ Github: https://github.com/teddylee777', '✔️ 코드스테이츠 X 테디노트 - 깃헙 블로그 제작하기 강의. ✔️ SK그룹 - 2023년 상반기 텐서플로우 딥러닝 과정 강의. ✔️ S-Oil - 파이썬 데이터 분석, 머신러닝 과정 강의 ...', 'Few Shot Prompt Template · yaml 형태로 관리하는 것을 추천! · 나중에 프롬프트 유지보수 관리에 용이 · 파일명을 신경써서 작성하기.', '데이터 분석, 머신러닝, 딥러닝, LLM 에 대한 내용을 다룹니다. 연구보다는 개발에 관심이 많습니다 \u200d♂️ ...more ...more fastcampus.co.kr/data_online_teddyand 2 ...', '안녕하세요? 두 번째 발표를 맡은 테디노트의 이경록(Teddy)입니다. 먼저, 귀중한 자리에 초청해 주신 랭체인 코리아 운영진 및 관계자 여러분께 감사 ...', '교육 스타트업 브레인크루 주식회사 설립, CEO. 특허출원 10-2019-0058830 (인공지능 기반 강의 추천 및 학습 동

In [71]:
' '.join(eval(search_result))

'테디노트 데이터와 인공지능을 좋아하는 개발자 노트 · 검색. 토글 메뉴. 카테고리 · 태그 · 연도 · 강의 · 어바웃미 · Teddy. Creator & Data Lover. 팔로우. Pangyo, ... 테디노트 X 패스트캠퍼스 "RAG 비법노트" · 환경 설정 (Mac) · 환경 설정 ... TeddyNote. Enter to select. to navigate. ESC to close. 테디노트.dev. Teddy Lee (이경록) ... 테디노트 TeddyNote. @teddynote. 23.3K subscribers•182 videos. 데이터 분석, 머신러닝, 딥러닝, LLM 에 대한 내용을 다룹니다. 연구보다는 개발에 관심이 많습니다 \u200d ... 테디노트 YouTube & Blog ▷ 랭체인 한국어 튜토리얼 저자(https://wikidocs.net/book/14314) ▷ Github: https://github.com/teddylee777 ✔️ 코드스테이츠 X 테디노트 - 깃헙 블로그 제작하기 강의. ✔️ SK그룹 - 2023년 상반기 텐서플로우 딥러닝 과정 강의. ✔️ S-Oil - 파이썬 데이터 분석, 머신러닝 과정 강의 ... Few Shot Prompt Template · yaml 형태로 관리하는 것을 추천! · 나중에 프롬프트 유지보수 관리에 용이 · 파일명을 신경써서 작성하기. 데이터 분석, 머신러닝, 딥러닝, LLM 에 대한 내용을 다룹니다. 연구보다는 개발에 관심이 많습니다 \u200d♂️ ...more ...more fastcampus.co.kr/data_online_teddyand 2 ... 안녕하세요? 두 번째 발표를 맡은 테디노트의 이경록(Teddy)입니다. 먼저, 귀중한 자리에 초청해 주신 랭체인 코리아 운영진 및 관계자 여러분께 감사 ... 교육 스타트업 브레인크루 주식회사 설립, CEO. 특허출원 10-2019-0058830 (인공지능 기반 강의 추천 및 학습 동기 부여를 제공하는 강의 서비스를 위해 전자 